# Intro 

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import schedule
import time
from csv import writer


In [2]:
url = 'https://www.blockchain.com/btc/unconfirmed-transactions'
r = requests.get(url)
soup = BeautifulSoup(r.text)


class="sc-1au2w4e-0 bTgHwk"

# Elements 

sc-1ryi78w-0.cILyoi.sc-16b9dsl-1.ZwupP.u3ufsr-0.eQTRKC

In [3]:
elements = soup.find_all(class_ = 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC')

# Hash

"sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK" 

In [4]:
hash = soup.find_all(class_ = 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK')

# OverviewList

In [5]:
overviewlist = []
for index, item in enumerate(hash):
    overviewlist.append([])
    overviewlist[index].append(hash[index].getText())

In [6]:
index =0
i =0
for item in enumerate(elements): 
    overviewlist[index].append(item[1].getText())    
    i+=1  
    if i==3: 
        index+=1
        i=0
        
    

# DataFrame

In [22]:
df = pd.DataFrame(overviewlist, columns =['Hash', 'Timestamp', 'BTC', 'USD'])
df['USD'] = df['USD'].str.replace('$', '',regex = True)
df['USD'] = df['USD'].str.replace(',', '',regex = True)
df['USD'] = df['USD'].astype(float)
df =df.sort_values(by='USD', ascending=False)
df_result =df.head(1)
df_result.head(1)

,Hash,Timestamp,BTC,USD
23,0a7be96e4ae54ee30e2d64965f886e09b9ad79442735b7...,16:06,12.47239108 BTC,590311.04


# Functions

In [3]:
df = pd.DataFrame(columns = ['Hash', 'Timestamp', 'BTC','USD'])
print(df)

Empty DataFrame
Columns: [Hash, Timestamp, BTC, USD]
Index: []


In [12]:
def addhash(hashelement,overviewlist):
    for index, item in enumerate(hashelement):
        overviewlist.append([])
        overviewlist[index].append(item.getText())
    return overviewlist

In [11]:
def addelements(elements,overviewlist): 
    index=0
    i=0
    for item in enumerate(elements):        
        overviewlist[index].append(item[1].getText())    
        i+=1  
        if i==3: 
            index+=1
            i=0

In [13]:
def createdataframe(overviewlist): 
    df= pd.DataFrame(overviewlist, columns =['Hash', 'Timestamp', 'BTC', 'USD']) 
    df['USD'] = df['USD'].str.replace('$', '',regex = True)
    df['USD'] = df['USD'].str.replace(',', '',regex =True)
    df['USD'] = df['USD'].astype(float)    
    '''with open('test.csv','a')as f: 
        writer_object = writer(f)
        writer_object.writerow(overviewlist)
        f.close()'''
    df.to_csv('test.csv',mode ='a',header = False, index =False)   
    return  df
    

In [15]:
def savedataframetocsv(timestamp,df): 
    df.to_csv(f'/home/udhaesel/Desktop/Dataframe/file{str(timestamp)}.csv', header = True,index=True)

In [16]:
def createcompletedataframe(dataframe,completedataframe):
    completedataframe = completedataframe.append(dataframe)
    
    return completedataframe


In [17]:
def updatecurrentoverview(newtimestamp):
     
     col_names =['Hash', 'Timestamp', 'BTC', 'USD']
   
     current_overview = pd.read_csv('test.csv',names=col_names)     
     currenttimestamp=current_overview.iloc[1]['Timestamp']      
     
     if newtimestamp != currenttimestamp:               
          rslt_df = current_overview.loc[current_overview['Timestamp']==currenttimestamp] 
          rslt_df =rslt_df.sort_values(by='USD', ascending=False)    
                  
          timestampresult =rslt_df.values.tolist()
          timestampresult = f"Timestamp: {timestampresult [0][1]} - Hash: { timestampresult [0][0]} - BTC : {timestampresult [0][2]} - USD: {timestampresult [0][3]}" 
          print("Saved to timesamp")
          overviewtimestamps =[]          
          if timestampresult[0][0] not in overviewtimestamps:
               with open('timestamps.txt', 'a') as f:
                    f.write('\n'+timestampresult )          
                    f.close()
               overviewtimestamps.append(timestampresult[0][0])
          newcurrentoverview =current_overview[current_overview['Timestamp'] != currenttimestamp]     
          newcurrentoverview.to_csv('test.csv',mode ='w',header = False,index =False)
          savedataframetocsv(newcurrentoverview.iloc[1]['Timestamp'],newcurrentoverview)
     else: 
          print("Scraping")



In [19]:
def scraper():    
    url = 'https://www.blockchain.com/btc/unconfirmed-transactions'
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    elements = soup.find_all(class_ = 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC')
    hashelement = soup.find_all(class_ = 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK')
    
    overviewlist = []
    addhash(hashelement,overviewlist)    
    addelements(elements, overviewlist)
    new_timestamp = overviewlist[0][1]
       
    
    #print(df_complete)     
    df_result= createdataframe(overviewlist)
    #print(df_result)
    
    #print(current_overview)    
    updatecurrentoverview(new_timestamp)
        
    '''final_result =df_result.values.tolist()
    savedataframetocsv(final_result[0][1],df)
    print(f"Timestamp: {final_result[0][1]} - Hash: { final_result[0][0]} - BTC : {final_result[0][2]} - USD: {final_result[0][3]}")'''

    

In [35]:
df_complete= pd.DataFrame(columns =['Hash', 'Timestamp', 'BTC', 'USD']) 
scraper(df_complete)


08:36
Scraping


# Schedule 

In [20]:
schedule.every(15).seconds.do(scraper)

while True: 
    schedule.run_pending()
    time.sleep(15)

Scraping
Scraping
Scraping
Saved to timesamp
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Saved to timesamp
Saved to timesamp
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Saved to timesamp
Saved to timesamp
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Saved to timesamp
Saved to timesamp
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Saved to timesamp
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Saved to timesamp
Scraping
Scraping
Scraping
Saved to timesamp
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Scraping
Saved to timesamp
Saved to timesamp


KeyboardInterrupt: 